# RePlay recommender models comparison

We will show the main RePlay functionality and compare performance of RePlay models on well-known MovieLens dataset. If you have not used RePlay before, start with 01_replay_basics.ipynb which introduces base concepts and describe main classes and functionality.

### Dataset
We will compare RePlay models on __MovieLens 1m__. 

### Dataset preprocessing: 
Ratings greater than or equal to 3 are considered as positive interactions.

### Data split
Dataset is split by date so that 20% of the last interactions as are placed in the test part. Cold items and users are dropped.

### Predict:
We will predict top-10 most relevant films for each user.

### Metrics
Quality metrics used:__ndcg@k, hitrate@k, map@k, mrr@k__ for k = 1, 5, 10
Additional metrics used: __coverage@k__ and __surprisal@k__.

In [1]:
! pip install rs-datasets

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%config Completer.use_jedi = False

In [4]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)

In [5]:
import logging
import time

from pyspark.sql import functions as sf, types as st
from pyspark.sql.types import IntegerType

from replay.preprocessing.data_preparator import DataPreparator, Indexer
from replay.metrics import Coverage, HitRate, MRR, MAP, NDCG, Surprisal, Experiment
from replay.models import (
    ItemKNN,
    SLIM, 
    PopRec, 
    RandomRec,
    UCB,
    Wilson, 
    Word2VecRec,
    ALSWrap,
)
from replay.experimental.models import (
    ADMMSLIM,
    LightFMWrap,
    MultVAE,
    NeuroMF,
)

from replay.models.base_rec import HybridRecommender
from replay.utils.session_handler import State
from replay.splitters import DateSplitter
from replay.utils import get_log_info
from rs_datasets import MovieLens

`State` object allows passing existing Spark session or create a new one, which will be used by the all RePlay modules.

To create session with custom parameters ``spark.driver.memory`` and ``spark.sql.shuffle.partitions`` use function `get_spark_session` from `session_handler` module.

In [6]:
spark = State().session
spark

22/07/06 12:26:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/06 12:26:02 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/07/06 12:26:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
spark.sparkContext.setLogLevel('ERROR')

In [8]:
logger = logging.getLogger("replay")

In [9]:
K = 10
K_list_metrics = [1, 5, 10]
BUDGET = 20
BUDGET_NN = 10
SEED = 12345

## 0. Preprocessing <a name='data-preparator'></a>

### 0.1 Data loading

In [10]:
data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


#### log preprocessing

- converting to spark dataframe
- renaming columns
- checking for nulls
- converting timestamp to Timestamp format

In [11]:
preparator = DataPreparator()

In [12]:
%%time
log = preparator.transform(columns_mapping={'user_id': 'user_id',
                                      'item_id': 'item_id',
                                      'relevance': 'rating',
                                      'timestamp': 'timestamp'
                                     }, 
                           data=data.ratings)

06-Jul-22 12:26:06, replay, INFO: Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.


CPU times: user 208 ms, sys: 28.4 ms, total: 236 ms
Wall time: 4.21 s


In [13]:
log.show(2)

+-------+-------+---------+-------------------+
|user_id|item_id|relevance|          timestamp|
+-------+-------+---------+-------------------+
|      1|   1193|      5.0|2000-12-31 22:12:40|
|      1|    661|      3.0|2000-12-31 22:35:09|
+-------+-------+---------+-------------------+
only showing top 2 rows



In [14]:
# will consider ratings >= 3 as positive feedback. A positive feedback is treated with relevance = 1
only_positives_log = log.filter(sf.col('relevance') >= 3).withColumn('relevance', sf.lit(1))
only_positives_log.count()

836478

In [15]:
# we will use only algorithms which do not require user and item features and thus set feature dataframes to None
user_features=None
item_features=None

<a id='indexing'></a>
### 0.2. Indexing

Convert given users' and items' identifiers (\_id) to integers starting at zero without gaps (\_idx) with Indexer class.

In [16]:
indexer = Indexer(user_col='user_id', item_col='item_id')

Take all available user and item ids from log and features and pass them to Indexer. The _ids_ could repeat, the indexes will be ordered by label frequencies, so the most frequent label gets index 0.

In [17]:
%%time
indexer.fit(users=log.select('user_id'),
           items=log.select('item_id'))

CPU times: user 30.9 ms, sys: 4.34 ms, total: 35.3 ms
Wall time: 1.62 s


In [ ]:
%%time
log_replay = indexer.transform(df=only_positives_log)
log_replay.show(2)

### 0.2. Data split

In [ ]:
# train/test split 
train_spl = DateSplitter(
    test_start=0.2,
    drop_cold_items=True,
    drop_cold_users=True,

)
train, test = train_spl.split(log_replay)
print('train info:\n', get_log_info(train))
print('test info:\n', get_log_info(test))

In [ ]:
train.is_cached

In [ ]:
# train/test split for hyperparameters selection
opt_train, opt_val = train_spl.split(train)
opt_train.count(), opt_val.count()

In [ ]:
opt_train.is_cached

In [ ]:
# negative feedback will be used for Wilson and UCB models
only_negatives_log = indexer.transform(df=log.filter(sf.col('relevance') < 3).withColumn('relevance', sf.lit(0.)))
test_start = test.agg(sf.min('timestamp')).collect()[0][0]

# train with both positive and negative feedback
pos_neg_train=(train
              .withColumn('relevance', sf.lit(1.))
              .union(only_negatives_log.filter(sf.col('timestamp') < test_start))
             )
pos_neg_train.cache()
pos_neg_train.count()

In [ ]:
pos_neg_train.is_cached

In [ ]:
train.show(2)

# 1. Metrics definition

In [ ]:
# experiment is used for metrics calculation
e = Experiment(test, {MAP(): K, NDCG(): K, HitRate(): K_list_metrics, Coverage(train): K, Surprisal(train): K, MRR(): K})

# 2. Models training

In [ ]:
def fit_predict_add_res(name, model, experiment, train, suffix=''):
    """
    Run fit_predict for the `model`, measure time on fit_predict and evaluate metrics
    """
    start_time=time.time()
    
    logs = {'log': train}
    predict_params = {'k': K, 'users': test.select('user_idx').distinct()}
    
    if isinstance(model, (Wilson, UCB)):
        logs['log'] = pos_neg_train

    if isinstance(model, HybridRecommender):
        logs['item_features'] = item_features
        logs['user_features'] = user_features
    
    predict_params.update(logs)

    model.fit(**logs)
    fit_time = time.time() - start_time

    pred=model.predict(**predict_params)
    pred.cache()
    pred.count()
    predict_time = time.time() - start_time - fit_time

    experiment.add_result(name + suffix, pred)
    metric_time = time.time() - start_time - fit_time - predict_time
    experiment.results.loc[name + suffix, 'fit_time'] = fit_time
    experiment.results.loc[name + suffix, 'predict_time'] = predict_time
    experiment.results.loc[name + suffix, 'metric_time'] = metric_time
    experiment.results.loc[name + suffix, 'full_time'] = (fit_time + 
                                                          predict_time +
                                                          metric_time)
    pred.unpersist()
    print(experiment.results[['NDCG@{}'.format(K), 'MRR@{}'.format(K), 'Coverage@{}'.format(K), 'fit_time']].sort_values('NDCG@{}'.format(K), ascending=False))

In [ ]:
def full_pipeline(models, experiment, train, suffix='', budget=BUDGET):
    """
    For each model:
        -  if required: run hyperparameters search, set best params and save param values to `experiment`
        - pass model to `fit_predict_add_res`        
    """
    
    for name, [model, params] in models.items():
        model.logger.info(msg='{} started'.format(name))
        if params != 'no_opt':
            model.logger.info(msg='{} optimization started'.format(name))
            best_params = model.optimize(opt_train, 
                                         opt_val, 
                                         param_borders=params, 
                                         item_features=item_features,
                                         user_features=user_features,
                                         k=K, 
                                         budget=budget)
            logger.info(msg='best params for {} are: {}'.format(name, best_params))
            model.set_params(**best_params)
        
        logger.info(msg='{} fit_predict started'.format(name))
        fit_predict_add_res(name, model, experiment, train, suffix)
        # here we call protected attribute to get all parameters set during model initialization
        experiment.results.loc[name + suffix, 'params'] = str(model._init_args)

## 2.1. Non-personalized models

In [ ]:
non_personalized_models = {'Popular': [PopRec(), 'no_opt'], 
          'Random (uniform)': [RandomRec(seed=SEED, distribution='uniform'), 'no_opt'], 
          'Random (popularity-based)': [RandomRec(seed=SEED, distribution='popular_based'), {"alpha": [-0.5, 100]}],
          'UCB': [UCB(exploration_coef=0.5), 'no_opt'],
          'Wilson': [Wilson(), 'no_opt']}

In [30]:
%%time
full_pipeline(non_personalized_models, e, train)

[I 2022-07-06 12:27:38,021] Trial 1 finished with value: 0.07162073312970957 and parameters: {'distribution': 'popular_based', 'alpha': 4.203306364179124}. Best is trial 1 with value: 0.07162073312970957.
[I 2022-07-06 12:27:45,266] Trial 2 finished with value: 0.05725865911574275 and parameters: {'distribution': 'popular_based', 'alpha': 59.7684614032647}. Best is trial 1 with value: 0.07162073312970957.
[I 2022-07-06 12:27:51,554] Trial 3 finished with value: 0.0525713357567491 and parameters: {'distribution': 'popular_based', 'alpha': 96.8635288390311}. Best is trial 1 with value: 0.07162073312970957.
[I 2022-07-06 12:27:58,045] Trial 4 finished with value: 0.05980595521579263 and parameters: {'distribution': 'popular_based', 'alpha': 68.9908834561353}. Best is trial 1 with value: 0.07162073312970957.
[I 2022-07-06 12:28:04,713] Trial 5 finished with value: 0.059186763359720417 and parameters: {'distribution': 'popular_based', 'alpha': 67.94812289677174}. Best is trial 1 with value:

                            NDCG@10    MRR@10  Coverage@10  fit_time
Popular                    0.243783  0.390426     0.033903  3.015610
Random (popularity-based)  0.065383  0.147795     0.755113  1.642121
Random (uniform)           0.021725  0.054846     0.957691  1.038643


06-Jul-22 12:30:09, replay, INFO: Wilson started                                
06-Jul-22 12:30:09, replay, INFO: Wilson fit_predict started


                            NDCG@10    MRR@10  Coverage@10  fit_time
Popular                    0.243783  0.390426     0.033903  3.015610
Random (popularity-based)  0.065383  0.147795     0.755113  1.642121
Random (uniform)           0.021725  0.054846     0.957691  1.038643
UCB                        0.000440  0.001239     0.019894  2.517043


06-Jul-22 12:30:10, replay, INFO: This model can't predict cold items, they will be ignored


                            NDCG@10    MRR@10  Coverage@10  fit_time
Popular                    0.243783  0.390426     0.033903  3.015610
Wilson                     0.092121  0.180976     0.017092  1.404204
Random (popularity-based)  0.065383  0.147795     0.755113  1.642121
Random (uniform)           0.021725  0.054846     0.957691  1.038643
UCB                        0.000440  0.001239     0.019894  2.517043
CPU times: user 2.71 s, sys: 900 ms, total: 3.61 s
Wall time: 3min 56s


In [31]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time,params
Popular,0.033903,0.284460,0.530290,0.645303,0.157301,0.390426,0.243783,0.118354,3.015610,8.435026,17.700714,29.151350,{'use_relevance': False}
Wilson,0.017092,0.083406,0.345040,0.414399,0.045002,0.180976,0.092121,0.262190,1.404204,7.008937,8.726699,17.139840,{'alpha': 0.05}
Random (popularity-based),0.755113,0.065847,0.259877,0.377524,0.025962,0.147795,0.065383,0.341928,1.642121,4.502160,7.376990,13.521271,"{'distribution': 'popular_based', 'alpha': 22...."
Random (uniform),0.957691,0.017559,0.100088,0.167691,0.007332,0.054846,0.021725,0.538677,1.038643,8.104163,8.470267,17.613074,"{'distribution': 'uniform', 'alpha': 0.0, 'see..."
UCB,0.019894,0.000878,0.001756,0.003512,0.000143,0.001239,0.000440,1.000000,2.517043,6.736516,7.048881,16.302440,{'coef': 0.5}


UCB is developed for iterative learning and improve its quality over time as the exploration reduces.

In [32]:

e.results.to_csv('res_21_rel_1.csv')

## 2.2  Personalized models without features

In [33]:
common_models = {
          'ADMM SLIM': [ADMMSLIM(seed=SEED), {"lambda_1": [1e-6, 10],
                                              "lambda_2": [1e-6, 1000]},],
          'Implicit ALS': [ALSWrap(seed=SEED), None], 
          'Explicit ALS': [ALSWrap(seed=SEED, implicit_prefs=False), None], 
          'ItemKNN': [ItemKNN(), None], 
          'LightFM': [LightFMWrap(random_state=SEED), {"no_components": [8, 512]}], 
          'SLIM': [SLIM(seed=SEED), None],
          'Word2Vec': [Word2VecRec(seed=SEED), None]
}

In [34]:
%%time
full_pipeline(common_models, e, train)

06-Jul-22 12:30:26, replay, INFO: ADMM SLIM started
06-Jul-22 12:30:26, replay, INFO: ADMM SLIM optimization started
[I 2022-07-06 12:30:26,340] A new study created in memory with name: no-name-c1b2e5b0-c114-41c0-9f61-80a6bb5f9f38
06-Jul-22 12:30:42, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:30:50,701] Trial 0 finished with value: 0.2117259515269491 and parameters: {'lambda_1': 6.19000824176862, 'lambda_2': 1.321380750470246}. Best is trial 0 with value: 0.2117259515269491.
06-Jul-22 12:30:57, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:31:10,990] Trial 1 finished with value: 0.1740469076951062 and parameters: {'lambda_1': 4.771025954858967e-06, 'lambda_2': 0.4168338462319922}. Best is trial 0 with value: 0.2117259515269491.
06-Jul-22 12:31:16, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:31:27,122] Trial 2 finished with value: 0.17069647570844562 and 

                            NDCG@10    MRR@10  Coverage@10   fit_time
Popular                    0.243783  0.390426     0.033903   3.015610
ADMM SLIM                  0.241104  0.407796     0.209022  18.283431
Wilson                     0.092121  0.180976     0.017092   1.404204
Random (popularity-based)  0.065383  0.147795     0.755113   1.642121
Random (uniform)           0.021725  0.054846     0.957691   1.038643
UCB                        0.000440  0.001239     0.019894   2.517043


/home/volodkevich/replay_tasks/notes_update_07_04/env39/lib/python3.9/site-packages/optuna/distributions.py:359: FutureWarning: Samplers and other components in Optuna will assume that `step` is 1. `step` argument is deprecated and will be removed in the future. The removal of this feature is currently scheduled for v4.0.0, but this schedule is subject to change.
  warnings.warn(
06-Jul-22 12:39:25, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 12:39:25, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:39:42,796] Trial 0 finished with value: 0.2087745613888222 and parameters: {'rank': 10}. Best is trial 0 with value: 0.2087745613888222.
06-Jul-22 12:39:45, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 12:39:45, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:40:02,078] Trial 1 finished with value: 0.20338683139989547 and parameters: {'

                            NDCG@10    MRR@10  Coverage@10   fit_time
Implicit ALS               0.253444  0.406855     0.131129   2.522428
Popular                    0.243783  0.390426     0.033903   3.015610
ADMM SLIM                  0.241104  0.407796     0.209022  18.283431
Wilson                     0.092121  0.180976     0.017092   1.404204
Random (popularity-based)  0.065383  0.147795     0.755113   1.642121
Random (uniform)           0.021725  0.054846     0.957691   1.038643
UCB                        0.000440  0.001239     0.019894   2.517043


06-Jul-22 12:48:08, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 12:48:08, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:48:25,372] Trial 0 finished with value: 0.008955289357265354 and parameters: {'rank': 10}. Best is trial 0 with value: 0.008955289357265354.
06-Jul-22 12:48:27, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 12:48:27, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:48:44,148] Trial 1 finished with value: 0.02269446649370753 and parameters: {'rank': 14}. Best is trial 1 with value: 0.02269446649370753.
06-Jul-22 12:49:21, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 12:49:21, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:49:39,444] Trial 2 finished with value: 0.026425074034488013 and parameters: {'rank': 204}. Best is trial 2 with valu

                            NDCG@10    MRR@10  Coverage@10   fit_time
Implicit ALS               0.253444  0.406855     0.131129   2.522428
Popular                    0.243783  0.390426     0.033903   3.015610
ADMM SLIM                  0.241104  0.407796     0.209022  18.283431
Wilson                     0.092121  0.180976     0.017092   1.404204
Random (popularity-based)  0.065383  0.147795     0.755113   1.642121
Explicit ALS               0.023355  0.055637     0.504063   2.654888
Random (uniform)           0.021725  0.054846     0.957691   1.038643
UCB                        0.000440  0.001239     0.019894   2.517043


[I 2022-07-06 12:57:29,730] Trial 0 finished with value: 0.20815145550561892 and parameters: {'num_neighbours': 10, 'shrink': 0}. Best is trial 0 with value: 0.20815145550561892.
06-Jul-22 12:57:29, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:57:37,865] Trial 1 finished with value: 0.2245591978916671 and parameters: {'num_neighbours': 39, 'shrink': 28}. Best is trial 1 with value: 0.2245591978916671.
06-Jul-22 12:57:37, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:57:46,011] Trial 2 finished with value: 0.2262232130605526 and parameters: {'num_neighbours': 87, 'shrink': 22}. Best is trial 2 with value: 0.2262232130605526.
06-Jul-22 12:57:46, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 12:57:54,095] Trial 3 finished with value: 0.22536674387760605 and parameters: {'num_neighbours': 23, 'shrink': 88}. Best is trial 2 with value: 0.2262232130605526.
06-Jul-22 

                            NDCG@10    MRR@10  Coverage@10   fit_time
ItemKNN                        0.257539  0.410211     0.053516   9.211113
Implicit ALS               0.253444  0.406855     0.131129   2.522428
Popular                    0.243783  0.390426     0.033903   3.015610
ADMM SLIM                  0.241104  0.407796     0.209022  18.283431
Wilson                     0.092121  0.180976     0.017092   1.404204
Random (popularity-based)  0.065383  0.147795     0.755113   1.642121
Explicit ALS               0.023355  0.055637     0.504063   2.654888
Random (uniform)           0.021725  0.054846     0.957691   1.038643
UCB                        0.000440  0.001239     0.019894   2.517043


06-Jul-22 13:00:34, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 13:00:34, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 13:00:51,717] Trial 0 finished with value: 0.18471621401739807 and parameters: {'loss': 'warp', 'no_components': 128}. Best is trial 0 with value: 0.18471621401739807.
06-Jul-22 13:00:54, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 13:00:54, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 13:01:10,432] Trial 1 finished with value: 0.20729766406398462 and parameters: {'loss': 'warp', 'no_components': 28}. Best is trial 1 with value: 0.20729766406398462.
06-Jul-22 13:01:14, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 13:01:14, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 13:01:30,872] Trial 2 finished with value: 0.17217865128124343 and para

In [35]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time,params
SLIM,0.063883,0.326602,0.585601,0.693591,0.176544,0.435288,0.271317,0.133530,11.034407,18.161200,7.238187,36.433794,"{'beta': 4.59012995343559, 'lambda_': 2.867711..."
LightFM,0.154665,0.312555,0.585601,0.696225,0.173850,0.430341,0.269468,0.167885,3.019165,15.975241,7.169686,26.164092,"{'no_components': 8, 'loss': 'warp', 'random_s..."
ItemKNN,0.053516,0.300263,0.560140,0.648815,0.168162,0.410211,0.257539,0.137523,9.211113,6.928585,7.640117,23.779816,"{'shrink': 100, 'use_relevance': False, 'num_n..."
Implicit ALS,0.131129,0.292362,0.562774,0.681299,0.162140,0.406855,0.253444,0.163824,2.522428,17.020579,7.487635,27.030642,"{'rank': 8, 'implicit_prefs': True, 'seed': 12..."
Popular,0.033903,0.284460,0.530290,0.645303,0.157301,0.390426,0.243783,0.118354,3.015610,8.435026,17.700714,29.151350,{'use_relevance': False}
ADMM SLIM,0.209022,0.284460,0.571554,0.676032,0.144341,0.407796,0.241104,0.192540,18.283431,6.070843,7.399216,31.753490,"{'lambda_1': 6.19000824176862, 'lambda_2': 1.3..."
Word2Vec (default),0.139255,0.147498,0.383670,0.500439,0.074579,0.247189,0.139835,0.237858,7.811991,79.425368,7.606133,94.843492,"{'rank': 100, 'window_size': 1, 'use_idf': Fal..."
Word2Vec (optimized),0.139255,0.147498,0.383670,0.500439,0.074579,0.247189,0.139835,0.237858,6.988796,77.598632,9.715263,94.302691,"{'rank': 100, 'window_size': 1, 'use_idf': Fal..."
Wilson,0.017092,0.083406,0.345040,0.414399,0.045002,0.180976,0.092121,0.262190,1.404204,7.008937,8.726699,17.139840,{'alpha': 0.05}
Random (popularity-based),0.755113,0.065847,0.259877,0.377524,0.025962,0.147795,0.065383,0.341928,1.642121,4.502160,7.376990,13.521271,"{'distribution': 'popular_based', 'alpha': 22...."


In [36]:
e.results.to_csv('res_22_rel_1.csv')

## 2.3 Neural models

In [37]:
nets = {
    'MultVAE (default)': [MultVAE(), 'no_opt'],
        'NeuroMF (default)': [NeuroMF(), 'no_opt'], 
        
        'MultVAE (optimized)': [MultVAE(), {"learning_rate": [0.001, 0.5],
                                   "dropout": [0, 0.5],
                                    "l2_reg": [1e-6, 5]
                                   }],
        'NeuroMF (optimized)': [NeuroMF(), {
                                    "learning_rate": [0.001, 0.5],
                                    "l2_reg": [1e-6, 5],
                                    "count_negative_sample": [1, 20]
                                    }]}

06-Jul-22 14:04:05, replay, INFO: The model is neural network with non-distributed training
06-Jul-22 14:04:05, replay, INFO: The model is neural network with non-distributed training
06-Jul-22 14:04:05, replay, INFO: The model is neural network with non-distributed training
06-Jul-22 14:04:05, replay, INFO: The model is neural network with non-distributed training


In [38]:
%%time
full_pipeline(nets, e, train, budget=BUDGET_NN)

06-Jul-22 14:04:05, replay, INFO: MultVAE (default) started
06-Jul-22 14:04:05, replay, INFO: MultVAE (default) fit_predict started
06-Jul-22 14:04:52, replay, INFO: This model can't predict cold items, they will be ignored
06-Jul-22 14:05:27, replay, INFO: NeuroMF (default) started                     
06-Jul-22 14:05:27, replay, INFO: NeuroMF (default) fit_predict started


                            NDCG@10    MRR@10  Coverage@10   fit_time
SLIM                       0.271317  0.435288     0.063883  11.034407
LightFM                    0.269468  0.430341     0.154665   3.019165
ItemKNN                        0.257539  0.410211     0.053516   9.211113
Implicit ALS               0.253444  0.406855     0.131129   2.522428
Popular                    0.243783  0.390426     0.033903   3.015610
ADMM SLIM                  0.241104  0.407796     0.209022  18.283431
Word2Vec (default)         0.139835  0.247189     0.139255   7.811991
Word2Vec (optimized)       0.139835  0.247189     0.139255   6.988796
MultVAE (default)          0.125255  0.243592     0.013449  46.468178
Wilson                     0.092121  0.180976     0.017092   1.404204
Random (popularity-based)  0.065383  0.147795     0.755113   1.642121
Explicit ALS               0.023355  0.055637     0.504063   2.654888
Random (uniform)           0.021725  0.054846     0.957691   1.038643
UCB             

06-Jul-22 14:11:01, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 14:11:01, replay, INFO: This model can't predict cold items, they will be ignored
06-Jul-22 14:12:14, replay, INFO: MultVAE (optimized) started                   
06-Jul-22 14:12:14, replay, INFO: MultVAE (optimized) optimization started
[I 2022-07-06 14:12:14,080] A new study created in memory with name: no-name-fa4b3560-c37c-4914-9b2f-8cd62680c044


                            NDCG@10    MRR@10  Coverage@10    fit_time
SLIM                       0.271317  0.435288     0.063883   11.034407
LightFM                    0.269468  0.430341     0.154665    3.019165
ItemKNN                        0.257539  0.410211     0.053516    9.211113
Implicit ALS               0.253444  0.406855     0.131129    2.522428
Popular                    0.243783  0.390426     0.033903    3.015610
ADMM SLIM                  0.241104  0.407796     0.209022   18.283431
NeuroMF (default)          0.221841  0.369230     0.245447  334.131373
Word2Vec (default)         0.139835  0.247189     0.139255    7.811991
Word2Vec (optimized)       0.139835  0.247189     0.139255    6.988796
MultVAE (default)          0.125255  0.243592     0.013449   46.468178
Wilson                     0.092121  0.180976     0.017092    1.404204
Random (popularity-based)  0.065383  0.147795     0.755113    1.642121
Explicit ALS               0.023355  0.055637     0.504063    2.654888
Ra

06-Jul-22 14:12:55, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 14:13:07,720] Trial 0 finished with value: 0.2163350086898685 and parameters: {'learning_rate': 0.02554053623172636, 'epochs': 100, 'latent_dim': 200, 'hidden_dim': 600, 'dropout': 0.3466404591450297, 'anneal': 0.1, 'l2_reg': 0.0002715786383841543, 'factor': 0.2, 'patience': 3}. Best is trial 0 with value: 0.2163350086898685.
06-Jul-22 14:13:47, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 14:13:58,205] Trial 1 finished with value: 0.1987445091995736 and parameters: {'learning_rate': 0.05633938909956612, 'epochs': 100, 'latent_dim': 200, 'hidden_dim': 600, 'dropout': 0.08000944046606179, 'anneal': 0.1, 'l2_reg': 0.022369772658029308, 'factor': 0.2, 'patience': 3}. Best is trial 0 with value: 0.2163350086898685.
06-Jul-22 14:14:37, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 14:14:46,915] Trial 2 fini

                            NDCG@10    MRR@10  Coverage@10    fit_time
SLIM                       0.271317  0.435288     0.063883   11.034407
LightFM                    0.269468  0.430341     0.154665    3.019165
ItemKNN                        0.257539  0.410211     0.053516    9.211113
Implicit ALS               0.253444  0.406855     0.131129    2.522428
Popular                    0.243783  0.390426     0.033903    3.015610
ADMM SLIM                  0.241104  0.407796     0.209022   18.283431
MultVAE (optimized)        0.236682  0.379818     0.033343   45.327343
NeuroMF (default)          0.221841  0.369230     0.245447  334.131373
Word2Vec (default)         0.139835  0.247189     0.139255    7.811991
Word2Vec (optimized)       0.139835  0.247189     0.139255    6.988796
MultVAE (default)          0.125255  0.243592     0.013449   46.468178
Wilson                     0.092121  0.180976     0.017092    1.404204
Random (popularity-based)  0.065383  0.147795     0.755113    1.642121
Ex

06-Jul-22 14:28:52, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 14:28:52, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 14:29:20,685] Trial 0 finished with value: 0.21474280262343395 and parameters: {'embedding_gmf_dim': 128, 'embedding_mlp_dim': 128, 'learning_rate': 0.009177256340761846, 'l2_reg': 0.8865722005650157, 'count_negative_sample': 4, 'factor': 0.2, 'patience': 3}. Best is trial 0 with value: 0.21474280262343395.
06-Jul-22 14:46:32, replay, INFO: This model can't predict cold users, they will be ignored
06-Jul-22 14:46:32, replay, INFO: This model can't predict cold items, they will be ignored
[I 2022-07-06 14:46:59,423] Trial 1 finished with value: 0.18552487113034768 and parameters: {'embedding_gmf_dim': 128, 'embedding_mlp_dim': 128, 'learning_rate': 0.010806082738783907, 'l2_reg': 1.990464309772189e-05, 'count_negative_sample': 15, 'factor': 0.2, 'patience': 3}. Best is trial 0 with value: 0.2

                            NDCG@10    MRR@10  Coverage@10    fit_time
SLIM                       0.271317  0.435288     0.063883   11.034407
LightFM                    0.269468  0.430341     0.154665    3.019165
ItemKNN                        0.257539  0.410211     0.053516    9.211113
Implicit ALS               0.253444  0.406855     0.131129    2.522428
Popular                    0.243783  0.390426     0.033903    3.015610
ADMM SLIM                  0.241104  0.407796     0.209022   18.283431
NeuroMF (optimized)        0.238649  0.379549     0.066405  566.323519
MultVAE (optimized)        0.236682  0.379818     0.033343   45.327343
NeuroMF (default)          0.221841  0.369230     0.245447  334.131373
Word2Vec (default)         0.139835  0.247189     0.139255    7.811991
Word2Vec (optimized)       0.139835  0.247189     0.139255    6.988796
MultVAE (default)          0.125255  0.243592     0.013449   46.468178
Wilson                     0.092121  0.180976     0.017092    1.404204
Ra

In [39]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time,params
SLIM,0.063883,0.326602,0.585601,0.693591,0.176544,0.435288,0.271317,0.133530,11.034407,18.161200,7.238187,36.433794,"{'beta': 4.59012995343559, 'lambda_': 2.867711..."
LightFM,0.154665,0.312555,0.585601,0.696225,0.173850,0.430341,0.269468,0.167885,3.019165,15.975241,7.169686,26.164092,"{'no_components': 8, 'loss': 'warp', 'random_s..."
ItemKNN,0.053516,0.300263,0.560140,0.648815,0.168162,0.410211,0.257539,0.137523,9.211113,6.928585,7.640117,23.779816,"{'shrink': 100, 'use_relevance': False, 'num_n..."
Implicit ALS,0.131129,0.292362,0.562774,0.681299,0.162140,0.406855,0.253444,0.163824,2.522428,17.020579,7.487635,27.030642,"{'rank': 8, 'implicit_prefs': True, 'seed': 12..."
Popular,0.033903,0.284460,0.530290,0.645303,0.157301,0.390426,0.243783,0.118354,3.015610,8.435026,17.700714,29.151350,{'use_relevance': False}
ADMM SLIM,0.209022,0.284460,0.571554,0.676032,0.144341,0.407796,0.241104,0.192540,18.283431,6.070843,7.399216,31.753490,"{'lambda_1': 6.19000824176862, 'lambda_2': 1.3..."
NeuroMF (optimized),0.066405,0.258999,0.547849,0.664618,0.149975,0.379549,0.238649,0.139592,566.323519,59.651768,10.435844,636.411131,"{'learning_rate': 0.037009681095325556, 'epoch..."
MultVAE (optimized),0.033343,0.280070,0.534680,0.640035,0.151451,0.379818,0.236682,0.121378,45.327343,17.850581,9.287374,72.465299,"{'learning_rate': 0.02554053623172636, 'epochs..."
NeuroMF (default),0.245447,0.248464,0.526778,0.647937,0.132896,0.369230,0.221841,0.205246,334.131373,60.435431,11.729832,406.296635,"{'learning_rate': 0.05, 'epochs': 20, 'embeddi..."
Word2Vec (default),0.139255,0.147498,0.383670,0.500439,0.074579,0.247189,0.139835,0.237858,7.811991,79.425368,7.606133,94.843492,"{'rank': 100, 'window_size': 1, 'use_idf': Fal..."


In [40]:
e.results.to_csv('res_23_rel_1.csv')

## 2.4 Models considering features

### 2.4.1 item features preprocessing

In [41]:
user_features=None

In [42]:
item_features_original = preparator.transform(columns_mapping={'item_id': 'item_id'}, 
                           data=data.items)
item_features = indexer.transform(df=item_features_original)
item_features.show(2)

06-Jul-22 16:52:33, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.


+--------+----------------+--------------------+
|item_idx|           title|              genres|
+--------+----------------+--------------------+
|      29|Toy Story (1995)|Animation|Childre...|
|     393|  Jumanji (1995)|Adventure|Childre...|
+--------+----------------+--------------------+
only showing top 2 rows



In [43]:
year = item_features.withColumn('year', sf.substring(sf.col('title'), -5, 4).astype(st.IntegerType())).select('item_idx', 'year')
year.show(2)

+--------+----+
|item_idx|year|
+--------+----+
|      29|1995|
|     393|1995|
+--------+----+
only showing top 2 rows



In [44]:
genres = (
    item_features.select(
        "item_idx",
        sf.split("genres", "\|").alias("genres")
    )
)

In [45]:
genres_list = (
    genres.select(sf.explode("genres").alias("genre"))
    .distinct().filter('genre <> "(no genres listed)"')
    .toPandas()["genre"].tolist()
)

In [46]:
genres_list

['Documentary',
 'Adventure',
 'Animation',
 'Comedy',
 'Thriller',
 'Sci-Fi',
 'Musical',
 'Horror',
 'Action',
 'Fantasy',
 'War',
 'Mystery',
 "Children's",
 'Drama',
 'Film-Noir',
 'Crime',
 'Western',
 'Romance']

In [47]:
item_features = genres
for genre in genres_list:
    item_features = item_features.withColumn(
        genre,
        sf.array_contains(sf.col("genres"), genre).astype(IntegerType())
    )
item_features = item_features.drop("genres").cache()
item_features.count()

3883

In [48]:
item_features = item_features.join(year, on='item_idx', how='inner')
item_features.cache()
item_features.count()

3883

In [49]:
item_features.show(2)

+--------+-----------+---------+---------+------+--------+------+-------+------+------+-------+---+-------+----------+-----+---------+-----+-------+-------+----+
|item_idx|Documentary|Adventure|Animation|Comedy|Thriller|Sci-Fi|Musical|Horror|Action|Fantasy|War|Mystery|Children's|Drama|Film-Noir|Crime|Western|Romance|year|
+--------+-----------+---------+---------+------+--------+------+-------+------+------+-------+---+-------+----------+-----+---------+-----+-------+-------+----+
|      29|          0|        0|        1|     1|       0|     0|      0|     0|     0|      0|  0|      0|         1|    0|        0|    0|      0|      0|1995|
|     393|          0|        1|        0|     0|       0|     0|      0|     0|     0|      1|  0|      0|         1|    0|        0|    0|      0|      0|1995|
+--------+-----------+---------+---------+------+--------+------+-------+------+------+-------+---+-------+----------+-----+---------+-----+-------+-------+----+
only showing top 2 rows



### 2.4.2 Models training

In [55]:
models_with_features = {'LightFM with item features': [LightFMWrap(random_state=SEED), {"no_components": [8, 128]}]}

In [56]:
%%time
full_pipeline(models_with_features, e, train, budget=BUDGET)

06-Jul-22 17:17:57, replay, INFO: LightFM with item features started
06-Jul-22 17:17:57, replay, INFO: LightFM with item features optimization started
[I 2022-07-06 17:17:57,541] A new study created in memory with name: no-name-2a8282b4-0780-4a15-be4f-8653b2d7239e
/home/volodkevich/replay_tasks/notes_update_07_04/env39/lib/python3.9/site-packages/optuna/distributions.py:359: FutureWarning: Samplers and other components in Optuna will assume that `step` is 1. `step` argument is deprecated and will be removed in the future. The removal of this feature is currently scheduled for v4.0.0, but this schedule is subject to change.
  warnings.warn(
06-Jul-22 17:18:46, replay, INFO: This model can't predict cold users, they will be ignored
[I 2022-07-06 17:19:08,098] Trial 0 finished with value: 0.19073469709711516 and parameters: {'loss': 'warp', 'no_components': 128}. Best is trial 0 with value: 0.19073469709711516.
06-Jul-22 17:19:42, replay, INFO: This model can't predict cold users, they wi

                             NDCG@10    MRR@10  Coverage@10    fit_time
SLIM                        0.271317  0.435288     0.063883   11.034407
LightFM                     0.269468  0.430341     0.154665    3.019165
ItemKNN                         0.257539  0.410211     0.053516    9.211113
Implicit ALS                0.253444  0.406855     0.131129    2.522428
LightFM with item features  0.249527  0.404767     0.211544   43.568975
Popular                     0.243783  0.390426     0.033903    3.015610
ADMM SLIM                   0.241104  0.407796     0.209022   18.283431
NeuroMF (optimized)         0.238649  0.379549     0.066405  566.323519
MultVAE (optimized)         0.236682  0.379818     0.033343   45.327343
NeuroMF (default)           0.221841  0.369230     0.245447  334.131373
Word2Vec (default)          0.139835  0.247189     0.139255    7.811991
Word2Vec (optimized)        0.139835  0.247189     0.139255    6.988796
MultVAE (default)           0.125255  0.243592     0.013449 

In [57]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time,params
SLIM,0.063883,0.326602,0.585601,0.693591,0.176544,0.435288,0.271317,0.133530,11.034407,18.161200,7.238187,36.433794,"{'beta': 4.59012995343559, 'lambda_': 2.867711..."
LightFM,0.154665,0.312555,0.585601,0.696225,0.173850,0.430341,0.269468,0.167885,3.019165,15.975241,7.169686,26.164092,"{'no_components': 8, 'loss': 'warp', 'random_s..."
ItemKNN,0.053516,0.300263,0.560140,0.648815,0.168162,0.410211,0.257539,0.137523,9.211113,6.928585,7.640117,23.779816,"{'shrink': 100, 'use_relevance': False, 'num_n..."
Implicit ALS,0.131129,0.292362,0.562774,0.681299,0.162140,0.406855,0.253444,0.163824,2.522428,17.020579,7.487635,27.030642,"{'rank': 8, 'implicit_prefs': True, 'seed': 12..."
LightFM with item features,0.211544,0.277436,0.582968,0.686567,0.155766,0.404767,0.249527,0.201322,43.568975,12.947130,10.765137,67.281242,"{'no_components': 53, 'loss': 'warp', 'random_..."
Popular,0.033903,0.284460,0.530290,0.645303,0.157301,0.390426,0.243783,0.118354,3.015610,8.435026,17.700714,29.151350,{'use_relevance': False}
ADMM SLIM,0.209022,0.284460,0.571554,0.676032,0.144341,0.407796,0.241104,0.192540,18.283431,6.070843,7.399216,31.753490,"{'lambda_1': 6.19000824176862, 'lambda_2': 1.3..."
NeuroMF (optimized),0.066405,0.258999,0.547849,0.664618,0.149975,0.379549,0.238649,0.139592,566.323519,59.651768,10.435844,636.411131,"{'learning_rate': 0.037009681095325556, 'epoch..."
MultVAE (optimized),0.033343,0.280070,0.534680,0.640035,0.151451,0.379818,0.236682,0.121378,45.327343,17.850581,9.287374,72.465299,"{'learning_rate': 0.02554053623172636, 'epochs..."
NeuroMF (default),0.245447,0.248464,0.526778,0.647937,0.132896,0.369230,0.221841,0.205246,334.131373,60.435431,11.729832,406.296635,"{'learning_rate': 0.05, 'epochs': 20, 'embeddi..."


Sometimes LightFM without features works better. It is our case.

In [58]:
e.results.to_csv('res_24_rel_1.csv')

# 3. Results

The best results by quality and time were shown by the commonly-used models such as ItemKNN, SLIM and LightFM and ALS

In [59]:
e.results.sort_values('NDCG@10', ascending=False).head(5)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time,params
SLIM,0.063883,0.326602,0.585601,0.693591,0.176544,0.435288,0.271317,0.133530,11.034407,18.161200,7.238187,36.433794,"{'beta': 4.59012995343559, 'lambda_': 2.867711..."
LightFM,0.154665,0.312555,0.585601,0.696225,0.173850,0.430341,0.269468,0.167885,3.019165,15.975241,7.169686,26.164092,"{'no_components': 8, 'loss': 'warp', 'random_s..."
ItemKNN,0.053516,0.300263,0.560140,0.648815,0.168162,0.410211,0.257539,0.137523,9.211113,6.928585,7.640117,23.779816,"{'shrink': 100, 'use_relevance': False, 'num_n..."
Implicit ALS,0.131129,0.292362,0.562774,0.681299,0.162140,0.406855,0.253444,0.163824,2.522428,17.020579,7.487635,27.030642,"{'rank': 8, 'implicit_prefs': True, 'seed': 12..."
LightFM with item features,0.211544,0.277436,0.582968,0.686567,0.155766,0.404767,0.249527,0.201322,43.568975,12.947130,10.765137,67.281242,"{'no_components': 53, 'loss': 'warp', 'random_..."
